In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import subprocess
from scipy.stats import chisquare
import os

# import scipy.stats as stats

In [47]:
d_domains = "../outputs/mutations/domains_expanded_iWES_v2_variants_snv_classified/"
files = os.listdir(d_domains)
ENST_codes = [f.replace(".bed", "") for f in files]

uniprotID_ENST_mapping = pd.read_csv("../../data/SFARI_TFs_with_ENST.csv")
uniprotID_ENST_mapping = uniprotID_ENST_mapping[["uniprotID", "ENST"]]
uniprotID_ENST_mapping["ENST"] = uniprotID_ENST_mapping["ENST"].str.split(".").str[0]
uniprotID_ENST_mapping_dict= dict(zip(uniprotID_ENST_mapping["ENST"], uniprotID_ENST_mapping["uniprotID"]))
uniprotID_ENST_mapping_dict['ENST00000434704'] = 'O60479'

In [106]:
# Uses bedtools jaccard function to get nucleotide length of CDS
# def return_cds_len(folder, code):
#     subprocess.call("bedtools jaccard -a ../outputs/mutations/" + folder + "/sorted/" + code + ".bed \
#     -b ../outputs/mutations/" + folder + "/sorted/" + code + ".bed \
#     > ../outputs/mutations/" + folder + "/lengths/" + code + ".bed", shell = True) 
#     cds_len = pd.read_csv("../outputs/mutations/" + folder + "/lengths/" + code + ".bed", sep = "\t")["intersection"][0]
#     return cds_len

def return_cds_len(folder, code, domain = None):
    loc_df = pd.read_csv("../outputs/mutations/" + folder + "/" + code, sep = "\t", header = None)
    if domain != None:
        loc_df = loc_df[loc_df[3] == domain]
    return sum(loc_df[2] - loc_df[1])

def sort_folder(folder):
    directory = "../outputs/mutations/" + folder 
    files = os.listdir(directory)
    i = 1
    for file in files:  
        if os.path.isdir(directory + "/" + file):
            # skip directories
            continue
        # print(i, file) # gene.bed
        name = file.split(".")[0] # gene
        # print("bedtools sort -i " + directory + "/" + file + " > ../outputs/mutations/cds_bed_format_sorted/" + name + ".bed")
        subprocess.call("bedtools sort -i " + directory + "/" + file + " > " + directory + "/sorted/" + name + ".bed", shell = True)
        i+=1

In [141]:
def generate_row(ENST_code, uniprotID_ENST_mapping_dict, min_var_freq_threshold, max_var_freq_threshold, domain_types, i = 1):
    tf_lengths, uniprotIDs, AD_lengths, cds_no_syns, AD_no_syns = [],[],[],[],[]
    DBD_lengths, DBD_no_syns = [], []
    lengths = []
    domain_lengths = {}
    domain_counts = {}
    
    for domain_type in domain_types:
        domain_lengths[domain_type] = []
    
    # Building a dataframe
    # for ENST_code in ENST_codes:
    print(i, ENST_code)
    i += 1

    # Sort domains
    sort_folder("domains_bed_format")

    # Get uniprotID
    uniprotID = uniprotID_ENST_mapping_dict[ENST_code]
    uniprotIDs.append(uniprotID)

    # Get TF length
    tf_lengths.append(return_cds_len("cds_bed_format", ENST_code))

    # Get sum of domain lengths
    for domain_type in domain_types:
        domain_lengths[domain_type].append(return_cds_len("domains_bed_format", uniprotID, domain_type))

    # Count # rows that are non-syn and below variant threshold in CDS clinvar annotated
    cds_clinvar = pd.read_csv("../outputs/mutations/cds_expanded_iWES_v2_variants_snv_classified/" + ENST_code + ".bed", sep = "\t", header = None)
    cds_no_syn_and_below_count = len(cds_clinvar[(cds_clinvar[14] == "No-Syn") 
                                                 & (cds_clinvar[11] > min_var_freq_threshold)
                                                 & (cds_clinvar[11] <= max_var_freq_threshold)])
    cds_no_syns.append(cds_no_syn_and_below_count)

    # Count # rows that are non-syn and below variant threshold per domain type
    domains_clinvar = pd.read_csv("../outputs/mutations/domains_expanded_iWES_v2_variants_snv_classified/" + ENST_code + ".bed", sep = "\t", header = None)
    domains_no_syn_and_below = domains_clinvar[(domains_clinvar[22] == "No-Syn") 
                                               & (domains_clinvar[19] > min_var_freq_threshold)
                                               & (domains_clinvar[19] <= max_var_freq_threshold)]

    for domain_type in domain_types:
        relev_rows = domains_no_syn_and_below[domains_no_syn_and_below[3] == domain_type]
        domain_counts[domain_type] = len(relev_rows)

    results = pd.DataFrame(data = {"uniprotID": uniprotIDs,
                           "TF_cds_length": tf_lengths,
                          "TF_missense" : cds_no_syns 
                          })

    for domain_type in domain_types:
        results[domain_type + "_cds_length"] = domain_lengths[domain_type]
        results[domain_type + "_missense"] = domain_counts[domain_type]

    return results

def generate_df(ENST_codes, uniprotID_ENST_mapping_dict, min_var_freq_threshold, max_var_freq_threshold, domain_types):
    i = 1
    rows = []
    for ENST_code in ENST_codes:
        rows.append(generate_row(ENST_code, uniprotID_ENST_mapping_dict, min_var_freq_threshold, max_var_freq_threshold, domain_types, i = i))
        i += 1
    return pd.concat(rows).reset_index(drop = True)

In [142]:
results = generate_df(ENST_codes[:2], uniprotID_ENST_mapping_dict, 0, 1, domain_types = ["AD"])
results

1 ENST00000356073
2 ENST00000348066


,uniprotID,TF_cds_length,TF_missense,AD_cds_length,AD_missense
0,P15884,2004,133,483,25
1,O94983,3609,449,882,108


In [156]:
import scipy.stats as stats

def add_fisher_p_vals_vs_rest_of_TF(results, domain_type, control = "TF"):
    fisher_exact_p_vals = []

    for i in results.index:
        domain_var_nt = results[domain_type + "_missense"].iloc[i]
        domain_nt_len = results[domain_type + "_cds_length"].iloc[i]

        TF_var_nt = results[control + "_missense"].iloc[i]
        TF_nt_len = results[control + "_cds_length"].iloc[i]

        data = [[domain_var_nt, TF_var_nt - domain_var_nt],
               [domain_nt_len - domain_var_nt, TF_nt_len - TF_var_nt - domain_nt_len + domain_var_nt]]

        p_val = stats.fisher_exact(data)[1]
        fisher_exact_p_vals.append(p_val)

    results[domain_type + "_fisher_exact_p_vals"] = fisher_exact_p_vals
    results = results.sort_values(by = domain_type + "_fisher_exact_p_vals")
    results = results.reset_index(drop = True)
    results = results.reset_index()
    results

In [157]:
add_fisher_p_vals_vs_rest_of_TF(results, "AD")
results

,uniprotID,TF_cds_length,TF_missense,AD_cds_length,AD_missense,AD_fisher_exact_p_vals
0,P15884,2004,133,483,25,0.171988
1,O94983,3609,449,882,108,0.860458
